In [1]:
import fosforml
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [2]:
my_session.connection.database

'ASSORTMENT_PLANNING'

In [3]:
my_session.connection.schema

'CPG_BRONZE'

In [4]:
table_name = "ASSORTMENT_PLANNING.CPG_BRONZE.SALES_CLEAN"

In [ ]:
sf_df = my_session.sql("select * from {}".format(table_name))
type(sf_df)
df=sf_df.to_pandas()
